In [1]:
from keras import models, layers
from keras.applications import VGG16
from keras import Input
from keras.models import Model
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint
import os
import matplotlib.pyplot as plt
import numpy as np
import math
from keras.models import load_model
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import cv2
 
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
 
train_dir = os.path.join('./Dataset/Train')
val_dir = os.path.join('./Dataset/Validation')
test_dir = os.path.join('./Dataset/Test')

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir, batch_size=4, target_size=(220, 200), color_mode='rgb',class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, batch_size=3, target_size=(220, 200), color_mode='rgb',class_mode='categorical')
test_set = test_datagen.flow_from_directory(test_dir, target_size=(220, 200),batch_size=1, class_mode='categorical', save_to_dir='./' , save_prefix='test', save_format='png',class_mode='categorical')

input_tensor = Input(shape=(220, 200, 3), dtype='float32', name='input')

In [ ]:
pre_trained_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(220, 200, 3))
pre_trained_vgg.trainable = False
# pre_trained_vgg.summary()

In [1]:
additional_model = models.Sequential()
additional_model.add(pre_trained_vgg)
additional_model.add(layers.Flatten())
additional_model.add(layers.Dense(4096, activation='relu'))
additional_model.add(layers.Dense(2048, activation='relu'))
additional_model.add(layers.Dense(1024, activation='relu'))
additional_model.add(layers.Dense(6, activation='softmax'))

additional_model.summary()

In [ ]:
checkpoint = ModelCheckpoint(filepath='pretrained_VGG_weight.hdf5', 
            monitor='loss', 
            mode='min', 
            save_best_only=True)
 
additional_model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=2e-5), metrics=['acc'])


history = additional_model.fit_generator(train_generator, 
            steps_per_epoch=math.ceil(train_generator.n / train_generator.batch_size), 
            epochs=50, 
            validation_data=val_generator, 
            validation_steps=math.ceil(val_generator.n / val_generator.batch_size), 
            callbacks=[checkpoint])

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(1, len(acc) + 1)
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Loss')
plt.legend()
 
plt.show()

In [ ]:
additional_model = load_model('pretrained_VGG_weight.hdf5')

In [ ]:
images = []

img, label = test_set.next()
n_img = len(label)

base = cv2.cvtColor(img[0], cv2.COLOR_RGB2BGR)  # keras는 RGB, openCV는 BGR이라 변경함
for idx in range(n_img - 1):
    img2 = cv2.cvtColor(img[idx + 1], cv2.COLOR_RGB2BGR)
    base = np.hstack((base, img2))
images.append(base)
images = np.concatenate(images)
img_test_a = np.uint8(images*255) 
img = images[0]
# cv2.imwrite('./test_1.jpg',img_test_a)
for idx in range(len(images) - 1):
    img = np.vstack((img, images[idx + 1]))
    # img = np.hstack((img, images[idx + 1]))
# cv2.imshow('result', img)
# cv2.imwrite('./test.jpg',img)